### Import Dependencies

In [6]:
import zipfile
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

### Load Data

Data linked in README.

In [2]:
# Extract files
zip_file_path = '/content/stat-486-image-classification.zip'
extract_to_directory = '/content/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_directory)

# Load labels
labels = pd.read_csv('training_labels.csv')
images_dir = 'training/training'

# Add the directory to the filename ID for full path
labels['ID'] = labels['ID'].apply(lambda x: os.path.join(images_dir, x))

### Define Generators

First, I created an image data generator that randomly modifies images through various ways like rescaling, rotation, and magnification to diversify the limited training data available.

The training and validation generators then ensure memory efficient training by loading the images into batches and applying the data augments.

In [3]:
# Initialize the ImageDataGenerator with a validation split and image modification
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest')

# Create the training and validation generators
train_generator = datagen.flow_from_dataframe(
    dataframe=labels,
    directory=None,
    x_col='ID',
    y_col='target',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=labels,
    directory=None,
    x_col='ID',
    y_col='target',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 2591 validated image filenames belonging to 5 classes.
Found 863 validated image filenames belonging to 5 classes.


### Defining the Model

The input data passes through a sequence of layers. First, they pass through 5 concolutional layers, with each doubling in the number of 3x3 filters and using the ReLU activation function. These layers utilize `BatchNormalization` to scale the inputs of each batch to have a mean of 0 and a variance of 1. They also utilize 2x2 `MaxPooling2D` windows to reduce input dimensionality while preserving the most prominent features.

After passing through the convolutional layers, the input is flattened to one dimension before passing through the one-dimensional dense layer with all 512 neurons connected to the neurons of the flattened layer. This provides a comprehensive and dimension-reduced summary of the input. 50% of these neurons are set to 0 to prevent overfitting.

The results are passed to a final dense layer with 5 neurons equal to the number of flower types in the data set. The `softmax` activation function produces a multi-class probability distribution.

The model is compile with the `categorical_crossentropy` loss function and `Adam` optimizer.

During training, the validation loss is monitored. If loss does not improve after 5 epochs, the learning rate is reduced. If loss does not improve after 10 epochs, the previous best weights are restored and training is stopped.

In [4]:
# Model definition
model = Sequential([
    # Convolutional layer 1
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Convolutional layer 2
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Convolutional layer 3
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Convolutional layer 4
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Convolutional layer 5
    Conv2D(512, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    # Flatten the results to feed into a DNN
    Flatten(),

    # 512 neuron hidden layer
    Dense(512, activation='relu'),
    Dropout(0.5),

    # Output layer
    Dense(5, activation='softmax')
])

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

### Model Fit

I trained this model for 100 epochs.

In [11]:
# Fit model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=25,
    verbose=1,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/25
81/81 [==============================] - 612s 8s/step - loss: 0.3885 - accuracy: 0.8599 - val_loss: 0.6116 - val_accuracy: 0.8100 - lr: 2.0000e-04
Epoch 2/25
81/81 [==============================] - 603s 7s/step - loss: 0.3779 - accuracy: 0.8695 - val_loss: 0.6473 - val_accuracy: 0.8053 - lr: 2.0000e-04
Epoch 3/25
81/81 [==============================] - 600s 7s/step - loss: 0.3773 - accuracy: 0.8634 - val_loss: 0.5964 - val_accuracy: 0.8019 - lr: 2.0000e-04
Epoch 4/25
81/81 [==============================] - 602s 7s/step - loss: 0.3768 - accuracy: 0.8723 - val_loss: 0.5806 - val_accuracy: 0.8053 - lr: 2.0000e-04
Epoch 5/25
81/81 [==============================] - 623s 8s/step - loss: 0.3243 - accuracy: 0.8788 - val_loss: 0.5596 - val_accuracy: 0.8076 - lr: 2.0000e-04
Epoch 6/25
81/81 [==============================] - 604s 7s/step - loss: 0.3576 - accuracy: 0.8780 - val_loss: 0.6709 - val_accuracy: 0.7984 - lr: 2.0000e-04
Epoch 7/25
81/81 [==============================] - 

### Save and Load Model

Model can be saved and reloaded to train over multiple days, as your free daily TPU/GPU usage on Google Colab is limited.

In [13]:
# Save model
model.save('flower_classification.h5')

In [7]:
# Load model
model = load_model('flower_classification_model.h5')